In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
### Player Count
# Total Number of Players
player_count = purchase_data["SN"].nunique()
pd.DataFrame({"Total Players":[player_count]})

,Total Players
0,576


In [3]:
### Purchasing Analysis (Total)
# Number of Unique Items
unique_items = purchase_data["Item ID"].nunique()

# Average Purchase Price
avg_price = purchase_data["Price"].mean()

# Total Number of Purchases
purchase_number = purchase_data["Purchase ID"].nunique()

# Total Revenue
total_rev = purchase_data["Price"].sum()

pd.DataFrame({"Number of Unique Items":[unique_items],
              "Average Price":["$"+str(round(avg_price,2))],
              "Number of Purchases":[purchase_number],
              "Total Revenue":["$"+'{0:,.2f}'.format(total_rev)]})

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [4]:
### Gender Demographics
# Percentage and Count of Male Players
# Percentage and Count of Female Players
# Percentage and Count of Other / Non-Disclosed
remove_duplicates = purchase_data.drop_duplicates(["SN"])
count_by_gender=pd.DataFrame(remove_duplicates["Gender"].value_counts())
gender_percentages=round((count_by_gender/player_count)*100,2)

# Combine data
gender_summary = pd.concat([count_by_gender,gender_percentages],axis=1)

# Name columns
gender_summary.columns = ["Total Count","Percentage of Players"]
gender_summary

,Total Count,Percentage of Players
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


In [5]:
### Purchasing Analysis (Gender)
# The below each broken by gender

# Purchase Count
purchase_count=purchase_data.groupby(["Gender"]).count()["Purchase ID"]

# Average Purchase Price
avg_purchase_price=round(purchase_data.groupby(["Gender"]).mean()["Price"],2)

# Total Purchase Value
total_purchase_value=round(purchase_data.groupby(["Gender"]).sum()["Price"],2)

# Average Purchase Total per Person by Gender
avg_total_purchases = round(total_purchase_value / remove_duplicates.groupby(["Gender"]).count()["Purchase ID"],2)

# Combine data
gender_purch_summary = pd.concat([purchase_count,avg_purchase_price,total_purchase_value,avg_total_purchases],axis=1)

# Name columns
gender_purch_summary.columns = ["Purchase Count","Average Purchase Price","Total Purchase Value","Avg Total Purchase per Person"]

# Format
gender_purch_summary["Average Purchase Price"] = gender_purch_summary["Average Purchase Price"].map("${:.2f}".format)
gender_purch_summary["Total Purchase Value"] = gender_purch_summary["Total Purchase Value"].map("${0:,.2f}".format)
gender_purch_summary["Avg Total Purchase per Person"] = gender_purch_summary["Avg Total Purchase per Person"].map("${:.2f}".format)
gender_purch_summary

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [6]:
### Age Demographics
# The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
  # Purchase Count
  # Average Purchase Price
  # Total Purchase Value
  # Average Purchase Total per Person by Age Group

# defining the bins and the titles
age_bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
# separating into bins
remove_duplicates["Age Ranges"] = pd.cut(remove_duplicates["Age"],age_bins,labels=group_names)
# defining variables and calculating
age_demos_totals = remove_duplicates["Age Ranges"].value_counts()
age_demos_percents = age_demos_totals / player_count *100
# creating the dataframe
age_summary = pd.DataFrame({"Total Count":age_demos_totals, "Percentage of Players":age_demos_percents})
age_summary = age_summary.round(2)
# display
age_summary.sort_index()

C:\Users\SHT7249\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,Total Count,Percentage of Players
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


In [7]:
# establishing the table we're making
purchase_data["Age Ranges"] = pd.cut(purchase_data["Age"],age_bins,labels=group_names)
# performing calculations
avg_age = purchase_data.groupby(["Age Ranges"]).mean()["Price"].rename("Average Purchase Price")
age_purchase_total = purchase_data.groupby(["Age Ranges"]).sum()["Price"].rename("Total Purchase Value")
age_count = purchase_data.groupby(["Age Ranges"]).count()["Price"].rename("Purchase Count")
# other variable definitions
normalized_total = age_purchase_total / age_summary["Total Count"]
# creating the dataframe
age_data = pd.DataFrame({"Purchase Count": age_count, "Average Purchase Price": avg_age, "Total Purchase Value": age_purchase_total, "Normalized Totals": normalized_total})
# formatting
age_data["Average Purchase Price"] = age_data["Average Purchase Price"].map("${:,.2f}".format)
age_data["Total Purchase Value"] = age_data["Total Purchase Value"].map("${:,.2f}".format)
age_data ["Purchase Count"] = age_data["Purchase Count"].map("{:,}".format)
age_data["Normalized Totals"] = age_data["Normalized Totals"].map("${:,.2f}".format)
age_data = age_data.loc[:, ["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]
# display
age_data

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [15]:
### Top Spenders
# Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
  # SN
  # Purchase Count
  # Average Purchase Price
  # Total Purchase Value

user_total = purchase_data.groupby(["SN"]).sum()["Price"].rename("Total Purchase Value")
user_average = purchase_data.groupby(["SN"]).mean()["Price"].rename("Average Purchase Price")
user_count = purchase_data.groupby(["SN"]).count()["Price"].rename("Purchase Count")
# establishing DataFrame
user_data = pd.DataFrame({"Purchase Count": user_count,"Average Purchase Price": user_average, "Total Purchase Value": user_total})
# displaying the table
top_5 = user_data.sort_values("Total Purchase Value", ascending=False).head(5)
# formatting
top_5["Average Purchase Price"] = user_data["Average Purchase Price"].map("${:,.2f}".format)
top_5["Total Purchase Value"] = user_data["Total Purchase Value"].map("${:,.2f}".format)
top_5

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [16]:
### Most Popular Items
# Identify the 5 most popular items by purchase count, then list (in a table):
  # Item ID
  # Item Name
  # Purchase Count
  # Item Price
  # Total Purchase Value

    # extracting data from the read file
pop_items = purchase_data.loc[:,["Item ID", "Item Name", "Price"]]
# calculations
total_item_purchase = pop_items.groupby(["Item ID","Item Name"]).sum()["Price"].rename("Total Purchase Value")
average_item_purchase = pop_items.groupby(["Item ID","Item Name"]).mean()["Price"]
item_count = pop_items.groupby(["Item ID","Item Name"]).count()["Price"].rename("Purchase Count")
# displaying table
pop_items_pd = pd.DataFrame({"Total Purchase Value": total_item_purchase, "Item Price": average_item_purchase, "Purchase Count": item_count})
pop_items_pd = pop_items_pd.loc[:,["Purchase Count", "Item Price", "Total Purchase Value"]]
display_popular_item = pop_items_pd.sort_values("Purchase Count", ascending=False).head(5)
# cleaning/formatting
display_popular_item["Item Price"] = pop_items_pd["Item Price"].map("${:,.2f}".format)
display_popular_item["Purchase Count"] = pop_items_pd["Purchase Count"].map("{:,}".format)
display_popular_item["Total Purchase Value"] = pop_items_pd["Total Purchase Value"].map("${:,.2f}".format)
sum_pop_items = display_popular_item.loc[:,["Purchase Count", "Item Price", "Total Purchase Value"]]
sum_pop_items

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [17]:
### Most Profitable Items
# Identify the 5 most profitable items by total purchase value, then list (in a table):
  # Item ID
  # Item Name
  # Purchase Count
  # Item Price
  # Total Purchase Value
    
# display sorted table
pop_items_pd.sort_values("Total Purchase Value", ascending=False).head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,4.614615,59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.230000,50.76
82,Nirvana,9,4.900000,44.10
145,Fiery Glass Crusader,9,4.580000,41.22
103,Singed Scalpel,8,4.350000,34.80


In [ ]:
#As final considerations:
# You must use the Pandas Library and the Jupyter Notebook.
# You must submit a link to your Jupyter Notebook with the viewable Data Frames.
# You must include a written description of three observable trends based on the data.
# See [Example Solution](HeroesOfPymoli/HeroesOfPymoli_starter.ipynb) for a reference on expected format.


Heroes Of Pymoli Data Analysis

1. 84% of all players are male and that's sad

2. On average, users spend $4.00

3. At $50, the most profitable item in the game is the Oathbreaker, Last Hope of the Breaking Stormpurchase